<a href="https://colab.research.google.com/github/RicardoLeonCo/MachineLearningPortfolio/blob/main/Credit_Card_Fraud_Detection_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#STEP 1: IMPORTING PACKAGES

import pandas as pd # this package is for data processing
import numpy as np #library to work with arrays
import matplotlib.pyplot as plt #library to create visualizations in Python
from termcolor import colored as cl #tool for text customization
import itertools #library in Python consisting of multiple methods used in iterators

from sklearn.preprocessing import StandardScaler #Tool for data normalization/standardization
from sklearn.model_selection import train_test_split #function that splits data arrays into 2 subsets: training & testing
from sklearn.tree import DecisionTreeClassifier #Decision tree algorithm
from sklearn.neighbors import KNeighborsClassifier #algorithm that implements learning based on the k nearest neighbors
from sklearn.linear_model import LogisticRegression #Logistic regression algorithm (ML algorithm used to predict the probability of a categorical dependent variable)
from sklearn.svm import SVC #Stands for "Support Vector Classification", and it's an SVM algorithm
from sklearn.ensemble import RandomForestClassifier #Random forest tree algorithm
from xgboost import XGBClassifier #Machine Learning Algorithm.

from sklearn.metrics import confusion_matrix #evaluation metric
from sklearn.metrics import accuracy_score #evaluation metric
from sklearn.metrics import f1_score #evaluation metric

**Notes:**
- Iterators are objects that allow us to iterate over all the elements of a collection and return one element at a time.
- When a dataset contains variable that are in different scales, *StandardScaler* performs the task of **Standardization** so they have a common scale.
- **Support Vector Machines** are supervised learning methods used for classification.
- **Random Forest Classifier** creates a set of decision trees from a random subset of the training set.
- **XGBoost** is an implementation of gradient boosted decision trees designed for speed and performance.
- A **Confusion Matrix** is a summary of prediction results on a classification problem.
- Accuracy is simply a ratio of correctly predicted observations to the total observations.
- Also known as F-score, the **F1 Score** is a weighted average of the precision and recall score. Used as an evaluation metric, a high F-score is a sign of a well-performing model.

In [10]:
#Now I'm going to import my data from the Kaggle dataset.

df = pd.read_csv('creditcard.csv')
df.drop('Time', axis = 1, inplace = True)
df.head

<bound method NDFrame.head of               V1        V2        V3  ...       V28  Amount  Class
0      -1.359807 -0.072781  2.536347  ... -0.021053  149.62    0.0
1       1.191857  0.266151  0.166480  ...  0.014724    2.69    0.0
2      -1.358354 -1.340163  1.773209  ... -0.059752  378.66    0.0
3      -0.966272 -0.185226  1.792993  ...  0.061458  123.50    0.0
4      -1.158233  0.877737  1.548718  ...  0.215153   69.99    0.0
...          ...       ...       ...  ...       ...     ...    ...
213964 -0.313355  1.038943 -0.093972  ...  0.032668    9.12    0.0
213965  2.030213  0.285221 -1.938915  ... -0.068185    2.90    0.0
213966  0.142229  0.672251  0.983900  ...  0.289856    8.57    0.0
213967  2.144033 -1.953243 -0.578686  ... -0.033122   84.00    0.0
213968  2.157143 -0.687990 -2.032580  ...       NaN     NaN    NaN

[213969 rows x 30 columns]>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
cases = len(df)
nonfraud_count = len(df[df.Class == 0])
fraud_count = len(df[df.Class == 1])
fraud_percentage = round(fraud_count/nonfraud_count*100, 2)

print(cl('CASE COUNT', attrs = ['bold']))
print(cl('--------------------------------------------------', attrs= ['bold']))
print(cl('Total number of cases are {}'.format(cases), attrs = ['bold']))
print(cl('Number of Non-fraud cases are {}'.format(nonfraud_count), attrs = ['bold']))
print(cl('Percentage of fraud cases is {}'.format(fraud_percentage), attrs = ['bold']))
print(cl('--------------------------------------------------', attrs = ['bold']))

CASE COUNT
--------------------------------------------------
Total number of cases are 213969
Number of Non-fraud cases are 213570
Percentage of fraud cases is 0.19
--------------------------------------------------


**Notes:**
- When working with the **print statement** I am wondering... *What does the "cl" stand for?*

In [13]:
nonfraud_cases = df[df.Class == 0]
fraud_cases = df[df.Class == 1]

#What are we doing here? Are we creating a class or some sort
#of boolean?

print(cl('CASE AMOUNT STATISTICS', attrs = ['bold']))
print(cl('--------------------------------------------------', attrs= ['bold']))
print(cl('NON-FRAUD CASE AMOUNT STATS', attrs= ['bold']))
print(nonfraud_cases.Amount.describe())
print(cl('--------------------------------------------------', attrs= ['bold']))
print(cl('FRAUD CASE AMOUNT STATS', attrs= ['bold']))
print(fraud_cases.Amount.describe())
print(cl('--------------------------------------------------', attrs= ['bold']))

#I think what this is doing is to call the describe function 
#with amounts. But what does *.amount* do?

CASE AMOUNT STATISTICS
--------------------------------------------------
NON-FRAUD CASE AMOUNT STATS
count    213570.000000
mean         90.156783
std         248.673400
min           0.000000
25%           6.000000
50%          23.165000
75%          79.640000
max       19656.530000
Name: Amount, dtype: float64
--------------------------------------------------
FRAUD CASE AMOUNT STATS
count     398.000000
mean      123.841307
std       257.528158
min         0.000000
25%         1.000000
50%        13.385000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64
--------------------------------------------------


In [14]:
#Now we are going to normalize our data because we need to
#reduce the wide range of values in the "Amount" variable to
#a smaller scale so we can work better.

sc = StandardScaler()
amount = df['Amount'].values

df['Amount'] = sc.fit_transform(amount.reshape(-1, 1))

print(cl(df['Amount'].head(10), attrs = ['bold']))

0    0.238851
1   -0.351958
2    1.159825
3    0.133822
4   -0.081343
5   -0.348017
6   -0.342709
7   -0.198716
8    0.011985
9   -0.347977
Name: Amount, dtype: float64


In [15]:
# DATA SPLIT
#Defining the independent and dependent variables.

X = df.drop('Class', axis = 1).values
y = df['Class'].values

#What does this line of code below mean?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

print(cl('X_train samples : ', attrs= ['bold']), X_train[:1])
print(cl('X_test samples : ', attrs= ['bold']), X_test[0:1])
print(cl('y_train samples : ', attrs=['bold']), y_train[0:20])
print(cl('y_test samples: ', attrs = ['bold']), y_test[0:20])

X_train samples :  [[-3.64971301  1.03705813 -2.61098587 -0.24862292  0.40450725 -2.17045741
   0.51355845  1.05182087 -0.9940041  -0.80980017 -1.34569008  0.91219019
   0.61394201  1.79762039 -0.51427845 -0.5812453   0.48558451 -0.44031489
  -0.1857532  -0.28375743  0.38628129  0.93250574 -1.09799487  0.12276579
   1.07470962  0.12621333  0.09688667 -0.35373672 -0.20193331]]
X_test samples :  [[-0.55972084  1.28998642 -0.10024787 -0.70158847  0.10509065 -0.91513154
   0.56998271  0.34510566 -0.36110484 -0.17027319  0.81275162  1.28472114
   0.4543221   0.46342535 -1.18854628  0.15344017 -0.49328117 -0.32174108
   0.13251209  0.01514434 -0.1857103  -0.43463981  0.1250484   0.07906838
  -0.44060151  0.12049292  0.22733596  0.08681519 -0.35481244]]
y_train samples :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
y_test samples:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


We have all the required components to build our classification models, which is our next step.

# **Building the Model**

For this project we will build 6 different types of classification models using algorithms by scikit-learn.

1. Decision Tree
2. K-Nearest Neighbors (KNN)
3. Logistic Regression
4. Support Vector Machine (SVM)
5. Random Forest
6. XGBoost

# **Evaluation Metrics for Classification Models**

We will use the following metrics to evaluate the models and decide which one is the best#

+ Accuracy Score
+ F1 Score
+ Confusion Matrix



In [16]:
# Models

# 1. Decision Tree

#When max_depth = 4 we allow the tree to split four times
#Criterion is a parameter that measures the quality of a split
#in our decision trees. "Gini Index" & "Entropy" are two
#different measures of impurity or disorder*?
tree_model = DecisionTreeClassifier(max_depth = 4, criterion = 'entropy')
tree_model.fit(X_train, y_train)
tree_yhat = tree_model.predict(X_test)

ValueError: ignored

In [ ]:
#Let's print the evaluation metrics for this model.

# 1. Accuracy score

print(cl('ACCURACY SCORE', attrs = ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))
print(cl('Accuracy score of the Decision Tree Model is {}'.format(accuracy_score(y_test, tree_yhat)), attrs= ['bold']))
print(cl('------------------------------------------------------------------------', attrs = ['bold']))

In [ ]:
#Models

#2. K-Nearest Neighbors

n = 5

knn = KNeighborsClassifier(n_neighbors = n)
knn.fit(X_train, y_train)
knn_yhat = knn.predict(X_test)

In [ ]:
#Evaluation Metrics from this metric

print(cl('Accuracy score of the KNN model is {}'.format(accuracy_score(y_test, knn_yhat)), attrs = ['bold'], color = 'green'))


In [ ]:
#Models

#3. Logistic Regression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_yhat = lr.predict(X_test)

In [ ]:
print(cl('Accuracy score of the Logistic Regression model is {}'.format(accuracy_score(y_test, lr_yhat)), attrs= ['bold']))

In [ ]:
#Models

#4. SVM

svm = SVC()
svm.fit(X_train, y_train)
svm_yhat = svm.predict(X_test)

In [ ]:
print(cl('Accuracy score of the SVM model is{}'.format(accuracy_score(y_test, svm_yhat)), attrs = ['bold']))

In [ ]:
#Models

#5. Random Forest Tree

rf = RandomForestClassifier(max_depth = 4)
rf.fit(X_train, y_train)
rf_yhat = rf.predict(X_test)

In [ ]:
print(cl('Accuracy score of the Random Forest Tree model is {}'.format(accuracy_score(y_test, rf_yhat)), attrs = ['bold']))

In [ ]:
 #Models 

 #6. XGBoost

 xgb = XGBClassifier(max_depth = 4)
 xgb.fit(X_train, y_train)
 xgb_yhat = xgb.predict(X_test)

In [ ]:
print(cl('Accuracy score of the XGBoost model is {}'.format(accuracy_score(y_test, xgb_yhat)), attrs = ['bold']))

The model with the least accuracy score is the Logistic Regression model whereas the model with the highest accuracy score is the KNN.

Another accuracy metric widely used in evaluating models is the f1 score. You calculate it by dividing the product F1 score = 2((precision * recall) / (precision + recall))

In [ ]:
# 2. F1 scores

print(cl('F1 SCORES', attrs = ['bold']))

print(cl('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, tree_yhat)), attrs = ['bold']))
print(cl('F1 score of the KNN model is {}'.format(f1_score(y_test, knn_yhat)), attrs = ['bold'], color = 'green'))
print(cl('F1 score of the Logistic Regression model is {}'.format(f1_score(y_test, lr_yhat)), attrs = ['bold'], color = 'red'))
print(cl('F1 score of the SVM model is {}'.format(f1_score(y_test, svm_yhat)), attrs = ['bold']))
print(cl('F1 score of the Random Forest Tree model is {}'.format(f1_score(y_test, rf_yhat)), attrs = ['bold']))
print(cl('F1 score of the XGBoost model is {}'.format(f1_score(y_test, xgb_yhat)), attrs = ['bold']))

# **Confusion Matrix**

In [ ]:
#3. Confusion Matrix

#defining the plot function

def plot_confusion_matrix(cm, classes, title, normalize = False, cmap = plt.cm.Blues):
  title = 'Confusion Matrix of {}'.format(title)
  if normalize:
    cm = cm.astype(float) / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation = 'nearest', cmap = cmap)
    #cmap stands for color map and it's part of a library. Matplotlib?
    plt.title(title) #plt is the pyplot package that we imported at the beginning of our code.
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation = 45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment = 'center',
               color = 'white' if cm[i, j] > thresh else 'black')
      
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#Now that we have the code above we are going to compute the confusion matrix for the models

tree_matrix = confusion_matrix(y_test, tree_yhat, labels = [0, 1]) # Decision Tree
knn_matrix = confusion_matrix(y_test, knn_yhat, labels = [0, 1]) #K-Nearest Neighbors
lr_matrix = confusion_matrix(y_test, lr_yhat, labels = [0, 1]) #Logistic Regression
svm_matrix = confusion_matrix(y_test, svm_yhat, labels = [0, 1]) #Support Vector Machine
rf_matrix = confusion_matrix(y_test, rf_yhat, labels = [0, 1]) #Random Forest Tree
xgb_matrix = confusion_matrix(y_test, xgb_yhat, labels = [0, 1]) #XGBoost

#Plot each confusion matrix

plt.rcParams['figure.figsize'] = (6,6)

#1. Decision tree

tree_cm_plot = plot_confusion_matrix(tree_matrix,
                                     classes = ['Non-Default(0)', 'Default(1)'],
                                     normalize = False, title = 'Decision Tree')
plt.savefig('tree_cm_plot.png')
plt.show()

#2. K-Nearest Neighbors

knn_cm_plot = plot_confusion_matrix(knn_matrix,
                                    classes = ['Non-default(0)', 'Default(1)'],
                                    normalize = False, title = 'KNN')
plt.savefig('knn_cm_plot.png')
plt.show()

lr_cm_plot = plot_confusion_matrix(lr_matrix,
                                   classes = ['Non-Default(0)', 'Default(1)'],
                                   normalize = False, title = 'Logistic Regression')
plt.savefig('lr_cm_plot.png')
plt.show()

#3. Logistic Regression

lr_cm_plot = plot_confusion_matrix(lr_matrix,
                                   classes = ['Non-Default(0)', 'Default(1)'],
                                   normalize = False, title = 'Logistic Regression')
plt.savefig('lr_cm_plot.png')
plt.show()

#4. Support Vector Machine

svm_cm_plot = plot_confusion_matrix(lr_matrix,
                                    classes = ['Non-Default(0)', 'Default(1)'],
                                    normalize = False, title = 'Logistic Regression')
plt.savefig('lr_cm_plot.png')
plt.show()

#5. Random Forest Tree

rf_cm_plot = plot_confusion_matrix(rf_matrix,
                                   classes = ['Non-Default(0)', 'Default(1)'],
                                   normalize = False, title = 'Random Forest Tree')
plt.savefig('rf_cm_plot.png')
plt.show()

#6. XGBoost

xgb_cm_plot = plot_confusion_matrix(xgb_matrix,
                                    classes = ['Non-Default(0)', 'Default(1)'],
                                    normalize = False, title = 'XGBoost')
plt.savefig('xgb_cm_plot.png')
plt.show()